In [ ]:
from matplotlib import pyplot as plt
!pip install pandas
!pip install copy
!pip install operator
!pip install deap
!pip install uuid
!pip install unittest
!pip install configparser
!pip install scikit-learn
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
import statistics as state

In [ ]:
import pandas as pd
count = 1
data = pd.read_csv('all_data_1.csv')
data

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
print("length",len(data['smoking_status']))
for i in range(len(data['smoking_status'])):
    print("aaaa",data['smoking_status'].iloc[i],i)
    state=pd.isna(data['smoking_status'].iloc[i])
    print("state ",state)
    if state!=True:
        if( data['smoking_status'].iloc[i]=='never smoked'):
            data['smoking_status'].iloc[i]=0
        elif( data['smoking_status'].iloc[i]=='formerly smoked'):
            data['smoking_status'].iloc[i]=1
        elif( data['smoking_status'].iloc[i]=='smokes'):
            data['smoking_status'].iloc[i]=2






In [ ]:
data

In [ ]:

def remove_high_nan(df,ratio = 0.65):
    for i in df.columns:
        if df[i].isnull().mean() > ratio:
            df.drop(i,axis = 1,inplace = True)

print("*************************************",data.isnull().sum())
remove_high_nan(data)
data_2 = data.copy()
data.isnull().sum().plot(kind = 'bar',fontsize = 12,figsize = (12,4))
plt.title('Null values we Have to handle',fontsize = 18)

In [ ]:
nullindexList=[]
not_nullindexList=[]
for i in range(43400):
    if(pd.isna(data['smoking_status'][i])==True):
        nullindexList.append(i)
    else:
        not_nullindexList.append(i)

In [ ]:
def add_label(data,attr,name_notnan = 'Training', name_nan = 'Predict'):
    null_pos = data[attr][data[attr].isnull()].index
    data['Label'] = name_notnan
    data['Label'].iloc[null_pos] = name_nan


def mean_median_imputation(data, kind = 'mean'):
    if kind=='mean':
        return data.fillna(np.mean(data.dropna()))
    elif kind == 'median':
        return data.fillna(np.median(data.dropna()))


def mode_imputation(data):
    return data.fillna(stat.mode(data))

In [ ]:
from sklearn import preprocessing
S_all=data
label_encoder = preprocessing.LabelEncoder()
S_all['gender']= label_encoder.fit_transform(S_all['gender'])
S_all['ever_married']= label_encoder.fit_transform(S_all['ever_married'])
S_all['work_type']= label_encoder.fit_transform(S_all['work_type'])
S_all['Residence_type']= label_encoder.fit_transform(S_all['Residence_type'])
S_all['avg_glucose_level']= label_encoder.fit_transform(S_all['avg_glucose_level'])

data=S_all

In [ ]:
data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
def Miss_forest_impute(data, attrb, dtype):
    train_data, test_data = data[data['Label']=='Training'], data[data['Label']=='Predict']
    Y = data[[attrb]]
    X = data.drop(['Label',attrb],axis = 1)
    y_train = train_data[[attrb]]
    x_train = train_data.drop(['Label',attrb],axis = 1)
    y_test = test_data[[attrb]]
    x_test = test_data.drop(['Label',attrb],axis = 1)
    if dtype == 'Continious':
        model = RandomForestRegressor()
        model.fit(X,Y)
    elif dtype == 'Categorical':
        model = RandomForestClassifier()
        model.fit(X,Y)
    y_pred = model.predict(x_test)
    return y_pred, Y[data['Label']=='Predict'].index

In [ ]:
if 'id' in data.columns:
    data.drop('id',inplace = True,axis = 1)

print("datadatadatadataaaaa222222",data)# Handling Categorical Values
not_null_attr = data.isnull().sum()[data.isnull().sum()==0].index
print("*************************************",data.isnull().sum())# Not Null Attributes
null_attr = data.isnull().sum()[data.isnull().sum() != 0].index              # Null Attributes
attrbs = {}
njobs = 12
for i in null_attr:
    print("11..................")
    df = pd.concat((data[not_null_attr], data[i]), axis = 1)
    add_label(df,i)
    if i=='smoking_status':
        df[i] = mean_median_imputation(df[[i]],kind = 'mean')
        for job in range(njobs):
            value,index = Miss_forest_impute(df,i,dtype = 'Continious')
            df[i].iloc[index] = value
    else:
        df[i] = mode_imputation(df[i])

        for job in range(njobs):
            value,index = Miss_forest_impute(df,i,dtype = 'Categorical')
            df[i].iloc[index] = value
    attrbs[i] = df[i]
print("************attrbsattrbs",attrbs)

In [ ]:
if count:
    data_2.drop(null_attr,axis = 1,inplace=True)
    count = 0

data_final = pd.concat((data_2,pd.DataFrame(attrbs)),axis = 1)
data_final


In [ ]:

def remove_high_nan(df,ratio = 0.65):
    for i in df.columns:
        if df[i].isnull().mean() > ratio:
            df.drop(i,axis = 1,inplace = True)

remove_high_nan(data_final)
data_final.isnull().sum().plot(kind = 'bar',fontsize = 12,figsize = (12,4))
plt.title('Null values we Have to handle',fontsize = 18)

In [ ]:
data_final.to_csv("missforested_data.csv")